# Proyecto 1: Integración de modelos Naive Bayes y regresión logística multinomial para clasificación multiclase con evaluación

**Integrantes**:

| Apellidos y nombres | Código |
|---------------------|--------|
| Meza Rodriguez, Fiorella Ivonne | 20192730G |
| Murillo Dominguez, Paul Hans | 20193507J |



**Descripción**: Desarrolla un sistema de clasificación de texto que combine modelos generativos (Naive Bayes) y discriminativos (Regresión Logística Multinomial) para tareas de clasificación multiclase, como la categorización de noticias. Implementa técnicas de descenso de gradiente estocástico con mini-lotes y regularización para optimizar los modelos. Evalúa el rendimiento utilizando métricas como precisión, recall, medida F y realiza pruebas de significancia estadística, incluyendo la prueba bootstrap pareada.

**Resultados esperados**:
- Implementación funcional de Naive Bayes y Regresión Logística Multinomial.
- Sistema de clasificación que integra ambos modelos para mejorar la precisión.
- Optimización mediante descenso de gradiente estocástico con mini-lotes y regularización.
- Evaluación exhaustiva utilizando precisión, recall, medida F, y pruebas de significancia.
- Análisis comparativo entre modelos generativos y discriminativos.
- Documentación detallada de la implementación y los resultados.

**Entradas**:
- Conjunto de datos etiquetado para clasificación multiclase (e.g., Reuters News Dataset).
- Parámetros de optimización y regularización.
- Configuraciones para pruebas estadísticas.

**Salidas**:
- Modelos entrenados de Naive Bayes y Regresión Logística Multinomial.
- Reporte de métricas de evaluación (precisión, recall, F1) para cada modelo.
- Resultados de pruebas de significancia estadística.
- Visualizaciones de la convergencia del descenso de gradiente.
- Análisis interpretativo de los modelos.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss
from sklearn.ensemble import VotingClassifier
from sklearn.utils import resample
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")

## 1. Conjunto de datos etiquetado para clasificación multiclase

### 1.1. Descarga del conjunto de datos Reuters News Dataset

In [2]:
nltk.download('wordnet')
nltk.download('reuters')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Overglitch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Overglitch\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Overglitch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### 1.2. Carga y etiquetado de los documentos

In [3]:
def get_documents_and_categories(fileids):
    documents = [reuters.raw(file) for file in fileids]
    categories = [reuters.categories(file) for file in fileids]
    return documents, categories


train_fileids = [file for file in reuters.fileids() if file.startswith('training/')]
test_fileids = [file for file in reuters.fileids() if file.startswith('test/')]

train_documents, train_categories = get_documents_and_categories(train_fileids)
test_documents, test_categories = get_documents_and_categories(test_fileids)

### 1.3. Codificación de las etiquetas

In [4]:
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train_categories)
test_labels = mlb.transform(test_categories)

In [5]:
trainDf = pd.DataFrame({"content": train_documents})
testDf = pd.DataFrame({"content": test_documents})

### 1.4. Preprocesamiento de los documentos

In [6]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk

# Cargar el lematizador y el stemmer
wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Cargar las stopwords desde un archivo
stopwords = set(w.rstrip() for w in open('data/reuters/stopwords'))


def tokenize_lemma_stopwords(text):
    # Reemplazar saltos de línea por espacios
    text = text.replace("\n", " ")

    # Tokenizar el texto en palabras
    tokens = nltk.tokenize.word_tokenize(text.lower())

    # Filtrar tokens para mantener solo palabras alfabéticas
    tokens = [t for t in tokens if t.isalpha()]

    # Lematizar las palabras para obtener su forma base
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]

    # Aplicar stemming a las palabras
    tokens = [stemmer.stem(t) for t in tokens]

    # Eliminar palabras cortas (menos de 3 caracteres)
    tokens = [t for t in tokens if len(t) > 2]

    # Eliminar stopwords
    tokens = [t for t in tokens if t not in stopwords]

    # Unir los tokens en una cadena de texto limpia
    cleanedText = " ".join(tokens)

    return cleanedText


def dataCleaning(df):
    # Crear una copia del DataFrame original
    data = df.copy()

    # Aplicar la función de limpieza a la columna 'content'
    data["content"] = data["content"].apply(tokenize_lemma_stopwords)

    return data

FileNotFoundError: [Errno 2] No such file or directory: 'data/reuters/stopwords'

In [ ]:
cleanedTrainData = dataCleaning(trainDf)
cleanedTestData = dataCleaning(testDf)

### 1.5. Vectorización de los documentos

In [ ]:
vectorizer = TfidfVectorizer()
vectorised_train_documents = vectorizer.fit_transform(cleanedTrainData["content"])
vectorised_test_documents = vectorizer.transform(cleanedTestData["content"])

In [ ]:
# Suponiendo que `vectorizer` y `vectorised_train_documents` ya están definidos
features = vectorizer.get_feature_names_out()
frequencies = np.asarray(vectorised_train_documents.sum(axis=0)).flatten()

# Ordenar las características por frecuencia
sorted_indices = np.argsort(frequencies)[::-1]
sorted_features = features[sorted_indices]
sorted_frequencies = frequencies[sorted_indices]

# Seleccionar las 50 características más frecuentes
top_n = 50
top_features = sorted_features[:top_n]
top_frequencies = sorted_frequencies[:top_n]

# Crear la gráfica
plt.figure(figsize=(10, 8))
plt.barh(top_features, top_frequencies)
plt.xlabel('Frequency')
plt.title('Frequency Distribution of Top 50 tokens')
plt.gca().invert_yaxis()  # Invertir el eje y para que la característica más frecuente esté en la parte superior
plt.show()

## 2. Implementación de modelos de clasificación

### 2.1. Implementación las métricas de evaluación

In [ ]:
ModelsPerformance = {}

def metricsReport(modelName: str, test_labels, predictions) -> None:
    accuracy = accuracy_score(test_labels, predictions)

    macro_precision = precision_score(test_labels, predictions, average='macro')
    macro_recall = recall_score(test_labels, predictions, average='macro')
    macro_f1 = f1_score(test_labels, predictions, average='macro')
    hamLoss = hamming_loss(test_labels, predictions)
    
    print(f"------{modelName} Model Metrics-----")
    print(
        f"Accuracy: {accuracy:.4f}\nHamming Loss: {hamLoss:.4f}\n"
        f"Precision (Macro): {macro_precision:.4f}\n"
        f"Recall (Macro): {macro_recall:.4f}\n"
        f"F1-measure (Macro): {macro_f1:.4f}"
    )
    ModelsPerformance[modelName] = macro_f1

### 2.2. Implementación de Naive Bayes

In [ ]:
naive_bayes_model = OneVsRestClassifier(MultinomialNB())
naive_bayes_model.fit(vectorised_train_documents, train_labels)

train_predictions = naive_bayes_model.predict(vectorised_train_documents)

metricsReport("Naive Bayes", train_labels, train_predictions)


### 2.3. Implementación de Regresión Logística Multinomial

In [ ]:
logistic_regression_model = OneVsRestClassifier(SGDClassifier(loss='log_loss', penalty='l2', alpha=1e-4, max_iter=1000, tol=1e-3))
logistic_regression_model.fit(vectorised_train_documents, train_labels)

train_predictions = logistic_regression_model.predict(vectorised_train_documents)

metricsReport("Logistic Regression", train_labels, train_predictions)